In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s5e11/sample_submission.csv
/kaggle/input/playground-series-s5e11/train.csv
/kaggle/input/playground-series-s5e11/test.csv


In [2]:
# Reading the training data as a pandas dataframe
train_df = pd.read_csv('/kaggle/input/playground-series-s5e11/train.csv')

In [3]:
# Getting a view of the dataframe
print(f'List of columns in the dataframe {train_df.columns}')
train_df.head()

List of columns in the dataframe Index(['id', 'annual_income', 'debt_to_income_ratio', 'credit_score',
       'loan_amount', 'interest_rate', 'gender', 'marital_status',
       'education_level', 'employment_status', 'loan_purpose',
       'grade_subgrade', 'loan_paid_back'],
      dtype='object')


,id,annual_income,debt_to_income_ratio,credit_score,loan_amount,interest_rate,gender,marital_status,education_level,employment_status,loan_purpose,grade_subgrade,loan_paid_back
0,0,29367.99,0.084,736,2528.42,13.67,Female,Single,High School,Self-employed,Other,C3,1.0
1,1,22108.02,0.166,636,4593.10,12.92,Male,Married,Master's,Employed,Debt consolidation,D3,0.0
2,2,49566.20,0.097,694,17005.15,9.76,Male,Single,High School,Employed,Debt consolidation,C5,1.0
3,3,46858.25,0.065,533,4682.48,16.10,Female,Single,High School,Employed,Debt consolidation,F1,1.0
4,4,25496.70,0.053,665,12184.43,10.21,Male,Married,High School,Employed,Other,D1,1.0


In [4]:
# Splitting the columns according to its type
# Dropping the id column since it won't add any relevance to predict output

unused_col = ['id', 'grade_subgrade']

numeric_col = ['debt_to_income_ratio', 'credit_score']

categorical_col = ['employment_status']

output_col = ['loan_paid_back']

imp_features = ['employment_status', 'debt_to_income_ratio', 'credit_score', 'loan_paid_back']

In [5]:
# Checking the null values
train_df.info()

# The data looks clean with no need of cleaning required

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 593994 entries, 0 to 593993
Data columns (total 13 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   id                    593994 non-null  int64  
 1   annual_income         593994 non-null  float64
 2   debt_to_income_ratio  593994 non-null  float64
 3   credit_score          593994 non-null  int64  
 4   loan_amount           593994 non-null  float64
 5   interest_rate         593994 non-null  float64
 6   gender                593994 non-null  object 
 7   marital_status        593994 non-null  object 
 8   education_level       593994 non-null  object 
 9   employment_status     593994 non-null  object 
 10  loan_purpose          593994 non-null  object 
 11  grade_subgrade        593994 non-null  object 
 12  loan_paid_back        593994 non-null  float64
dtypes: float64(5), int64(2), object(6)
memory usage: 58.9+ MB


In [6]:
# Obtaining the descriptive stats of different available columns
train_df.describe()

,id,annual_income,debt_to_income_ratio,credit_score,loan_amount,interest_rate,loan_paid_back
count,593994.000000,593994.000000,593994.000000,593994.000000,593994.000000,593994.000000,593994.000000
mean,296996.500000,48212.202976,0.120696,680.916009,15020.297629,12.356345,0.798820
std,171471.442235,26711.942078,0.068573,55.424956,6926.530568,2.008959,0.400883
min,0.000000,6002.430000,0.011000,395.000000,500.090000,3.200000,0.000000
25%,148498.250000,27934.400000,0.072000,646.000000,10279.620000,10.990000,1.000000
50%,296996.500000,46557.680000,0.096000,682.000000,15000.220000,12.370000,1.000000
75%,445494.750000,60981.320000,0.156000,719.000000,18858.580000,13.680000,1.000000
max,593993.000000,393381.740000,0.627000,849.000000,48959.950000,20.990000,1.000000


In [7]:
# Checking the data for different type of loan purposes
train_df['loan_paid_back'].value_counts()

loan_paid_back
1.0    474494
0.0    119500
Name: count, dtype: int64

In [8]:
train_df = train_df[imp_features]

In [9]:
# drop_first=True to avoid dummy trap
train_df_ohe = pd.get_dummies(train_df, columns=categorical_col, drop_first=False)  

In [10]:
train_df_ohe

,debt_to_income_ratio,credit_score,loan_paid_back,employment_status_Employed,employment_status_Retired,employment_status_Self-employed,employment_status_Student,employment_status_Unemployed
0,0.084,736,1.0,False,False,True,False,False
1,0.166,636,0.0,True,False,False,False,False
2,0.097,694,1.0,True,False,False,False,False
3,0.065,533,1.0,True,False,False,False,False
4,0.053,665,1.0,True,False,False,False,False
...,...,...,...,...,...,...,...,...
593989,0.152,703,1.0,True,False,False,False,False
593990,0.105,559,1.0,True,False,False,False,False
593991,0.072,675,1.0,True,False,False,False,False
593992,0.067,740,1.0,True,False,False,False,False


In [11]:
#train_df_ohe = train_df_ohe.drop(unused_col, axis=1)

In [12]:
train_df_ohe

,debt_to_income_ratio,credit_score,loan_paid_back,employment_status_Employed,employment_status_Retired,employment_status_Self-employed,employment_status_Student,employment_status_Unemployed
0,0.084,736,1.0,False,False,True,False,False
1,0.166,636,0.0,True,False,False,False,False
2,0.097,694,1.0,True,False,False,False,False
3,0.065,533,1.0,True,False,False,False,False
4,0.053,665,1.0,True,False,False,False,False
...,...,...,...,...,...,...,...,...
593989,0.152,703,1.0,True,False,False,False,False
593990,0.105,559,1.0,True,False,False,False,False
593991,0.072,675,1.0,True,False,False,False,False
593992,0.067,740,1.0,True,False,False,False,False


In [13]:
X_col = [x for x in train_df_ohe if x != 'loan_paid_back']
y_col = 'loan_paid_back'

In [14]:
from sklearn.model_selection import train_test_split, StratifiedKFold

X = train_df_ohe[X_col]
y = train_df_ohe[y_col]

# stratified train/test split (preserve class proportions)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

In [15]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression  # example estimator

# 3) Create ColumnTransformer - scale numeric cols, passthrough other columns
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_col),
    ],
    remainder='passthrough'   # keep one-hot and any other columns unchanged
)

# 4) Fit preprocessor on train, transform train & test
X_train_trans = preprocessor.fit_transform(X_train)
X_test_trans = preprocessor.transform(X_test)

# 5) (Optional) convert back to DataFrame with sensible column order/names
# ColumnTransformer orders features as: (all transformed outputs) + (passthrough cols in original order)
passthrough_cols = [c for c in X.columns if c not in numeric_col]
out_cols = numeric_col + passthrough_cols
X_train_trans_df = pd.DataFrame(X_train_trans, columns=out_cols, index=X_train.index)
X_test_trans_df  = pd.DataFrame(X_test_trans,  columns=out_cols, index=X_test.index)

# 6) Example: plug into a pipeline with an estimator (keeps scaling inside CV correctly)
pipeline = Pipeline([
    ('pre', preprocessor),
    ('clf', LogisticRegression(max_iter=2000))
])

pipeline.fit(X_train, y_train)
print("Train score:", pipeline.score(X_train, y_train))
print("Test score:", pipeline.score(X_test, y_test))

Train score: 0.8997422110922885
Test score: 0.8994351804308117


In [16]:
test_df = pd.read_csv('/kaggle/input/playground-series-s5e11/test.csv')

In [17]:
test_df = test_df[['employment_status', 'debt_to_income_ratio', 'credit_score']]

In [18]:
test_df_ohe = pd.get_dummies(test_df, columns=categorical_col, drop_first=False)
#test_df_ohe = test_df_ohe.drop(unused_col, axis=1)
test_df_ohe

,debt_to_income_ratio,credit_score,employment_status_Employed,employment_status_Retired,employment_status_Self-employed,employment_status_Student,employment_status_Unemployed
0,0.049,626,True,False,False,False,False
1,0.093,732,True,False,False,False,False
2,0.367,611,True,False,False,False,False
3,0.110,671,True,False,False,False,False
4,0.081,688,True,False,False,False,False
...,...,...,...,...,...,...,...
254564,0.068,744,True,False,False,False,False
254565,0.091,634,True,False,False,False,False
254566,0.096,718,True,False,False,False,False
254567,0.094,739,True,False,False,False,False


In [19]:
# get probability for positive class
probs = pipeline.predict_proba(test_df_ohe)[:, 1]

In [20]:
probs

array([0.9361737 , 0.97474865, 0.17711447, ..., 0.96815424, 0.97682449,
       0.85663666])

In [21]:
sub = pd.read_csv('/kaggle/input/playground-series-s5e11/sample_submission.csv')
sub[sub.columns[1]] = probs

In [22]:
test_df

,employment_status,debt_to_income_ratio,credit_score
0,Employed,0.049,626
1,Employed,0.093,732
2,Employed,0.367,611
3,Employed,0.110,671
4,Employed,0.081,688
...,...,...,...
254564,Employed,0.068,744
254565,Employed,0.091,634
254566,Employed,0.096,718
254567,Employed,0.094,739


In [23]:
sub

,id,loan_paid_back
0,593994,0.936174
1,593995,0.974749
2,593996,0.177114
3,593997,0.928460
4,593998,0.959794
...,...,...
254564,848558,0.984335
254565,848559,0.906478
254566,848560,0.968154
254567,848561,0.976824


In [24]:
sub.to_csv('submission_imp_features.csv', index=False)

In [25]:
# Post Submission Analysis
clf = pipeline.named_steps['clf']

In [26]:
# Binary
if clf.coef_.shape[0] == 1:
    coefs = clf.coef_.ravel()
    importance = pd.Series(np.abs(coefs), index=X.columns).sort_values(ascending=False)
    odds_ratio = pd.Series(np.exp(coefs), index=X.columns).sort_values(ascending=False)

print("Top features by absolute coefficient:\n", importance.head(50))

Top features by absolute coefficient:
 employment_status_Retired          4.929552
employment_status_Unemployed       3.791891
employment_status_Student          2.183029
employment_status_Employed         1.209097
employment_status_Self-employed    1.193535
debt_to_income_ratio               0.862711
credit_score                       0.795564
dtype: float64
